In [1]:
%cd /workspace/stk-analyst-chatbot
import json
with open('data/test.json','r')as f:
    multiturn_test = json.load(f)

/workspace/stk-analyst-chatbot


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
multiturn_test[0]

{'message': [{'role': 'system',
   'content': '당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하며, 각 함수의 사용 방법과 파라미터는 <tools></tools> XML 태그 안에 명시되어 있습니다. 함수를 호출할 때는 사용자가 제공한 값에 따라 정확히 실행해야 합니다.\n\n사용 가능한 함수와 각 파라미터의 설명은 다음과 같습니다:\n\n1. get_issues_summarized\n   - 설명: 특정 회사나 키워드에 대한 이슈를 검색하고 요약합니다.\n   - 파라미터:\n     - keyword: 검색하고자 하는 회사명 또는 키워드.\n     - days: 검색 기간(일 단위).\n\n2. get_reddit_hotissue\n   - 설명: 금융 시장에서 현재 주목받고 있는 이슈를 요약합니다.\n   - 파라미터:\n     - days: 검색 기간(일 단위).\n\n3. get_earnings\n   - 설명: 기업의 재무재표나 현금흐름을 가져오고 분석합니다. 이전 년도 데이터가 필요한 경우, 해당 데이터를 추가로 호출합니다.\n           함수 호출 시 연도와 분기 설정에 주의해야 합니다. 보통 1분기 실적발표는 4월 중순 ~ 5월 초, 2분기는 7월 중순 ~ 8월초, 3분기는 10월중순~ 11월 초, \n            4분기는 1월 중순 ~2월초에 발표 된다. \n            ## 예시: 1. 오늘 날짜 2025년 2월 18일 , 최근 실적발표의 year  과 quarter 은?\n            year : \'2024\' , quarter: -1 또는, 4 \n   - 파라미터:\n     - symbol: 데이터를 검색하고자 하는 기업 심볼.\n     - analysis_type: 분석 유형(growth, profitability, stability, valuation, cashflow, dividend, cost, NA).

In [14]:
!pip install vllm==0.6.0
!pip install scikit-learn
!pip install fitz
!pip install praw
!pip install yfinance
!pip install PyMuPDF


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [15]:
%pip install pandas
# import streamlit as st
import pandas as pd
# import fitz
import io
import time
import json
import os
import urllib.request
import praw
from datetime import datetime, timedelta
import yfinance as yf
import requests
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:

import string
from vllm import LLM, SamplingParams
# LLM

In [8]:
model_id1='output_dir/qwen2.5-7b-multiturn_fc/model1800'
llm1800 = LLM(model=model_id1)
tokenizer1800 = AutoTokenizer.from_pretrained(model_id1)

INFO 04-03 04:40:56 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='output_dir/qwen2.5-7b-multiturn_fc/model1800', speculative_config=None, tokenizer='output_dir/qwen2.5-7b-multiturn_fc/model1800', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=output_dir/qwen2.5-7b-multiturn_fc/model1800, use_v2_block_manager=False, num_schedu

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-03 04:41:25 model_runner.py:926] Loading model weights took 14.2487 GB
INFO 04-03 04:41:27 gpu_executor.py:122] # GPU blocks: 61092, # CPU blocks: 4681
INFO 04-03 04:41:32 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-03 04:41:32 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-03 04:41:45 model_runner.py:1335] Graph capturing finished in 12 secs.


In [17]:
tools = [{"type": "function",
       "function":   {
        "name":'get_earnings',
        "description":'당신은 기업의 실적, 재무재표 또는 현금흐름을 가져오고 분석하는 로봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "symbol":{'type':'string', "description": '실적 데이터를 찾고자 하는 기업의 symbol'},
                'analysis_type':{'type':'string', "description":"""기업의 재무 데이터에서 어떤 정보를 추출하고자 하는지 출력하는 파라미터입니다.
                매출 성장률, 순이익 성장률, 영업이익 성장률 과 같은 기업의 성장성을 요청할경우 'growth'라고 출력,
                얼마나 효율적으로 이익을 창출하는지 수익성과 관련한 질문일경우 'profitability' 출력,
                부채 수준과 재무 건전성과 관련한 질문일경우 'stability' 출력,
                현재 가치와 적정 주가와 관련한 질문일경우 'valuation' 출력,
                현금을 얼마나 잘 창출하는지 확인하여 재무건전성과 관련한 질문일경우 'cashflow' 출력,
                배당 정책과 주식 발행 내역과 관련한 질문일경우 'dividend' 출력,
                비용 절감 능력과 효율성과 관련한 질문일경우 'cost' 출력,
                그외의 재무,실적,현금흐름과 연관이있지만 파라미터를 찾을수없는경우 'NA'라고 출력합니다.

                """},
                "type_":{'type':'string', "description": "연간 데이터 일경우 'yearly' 아닐경우 'quarter' 이라고 표기"},
                "year":{'type':'string', "description": '데이터를 찾고자 하는 연도, 명시 하지 않을경우 가장 최근 연도 로 설정, 1월일경우 , 해당연도에 데이터가 입력되지않을수있음 따라서 최근연도-1 값을 연도라고 표기 '},
                "quarter":{'type':'string', "description": '데이터를 찾고자 하는 분기로 1~4로 이루어진 숫자, 마지막 또는 최근 분기일경우 또는 언급이 없을경우 -1 이라고 표기 '}
            },
            "required":['symbol', 'analysis_type','type_', 'year','quarter'],
            "additionalProperties": False
        },
             "strict": True
    }},
         {"type": "function",
       "function":   {
        "name":'get_consensus',
        "description":'당신은 기업의 (EPS) 컨센서스 데이터 또는, 매수/매도/홀드 의견 목표주가를 가져와서 분석하는 로봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "symbol":{'type':'string', "description": ' 데이터를 찾고자 하는 기업의 symbol'},
                "year":{'type':'string', "description": '데이터를 찾고자 하는 연도, 명시 하지 않을경우 가장 최근 연도 로 설정, 1월일경우 , 해당연도에 데이터가 입력되지않을수있음 따라서 최근연도-1 값을 연도라고 표기 '},
                "quarter":{'type':'string', "description": '데이터를 찾고자 하는 분기로 1~4로 이루어진 숫자, 최근 또는 마지막일경우 -1 이라고 표기 '}
            },
            "required":['symbol', 'year','quarter'],
            "additionalProperties": False
        },
             "strict": True
    }},
        {"type": "function",
         "function":    {
        "name":'get_issues_summarized',
        "description":'당신은 특정 기업 또는 키워드의 이슈검색 챗봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "keyword":{'type':'string', "description": '이슈/현황을 검색하고자 하는 회사명 또는 키워드 in English '},
                "days":{'type':'integer', "description": '검색하고자 하는 기간 예시: 하루, 일주일, 한달, 등등'}

            },
            "required":['keyword','days'],
            "additionalProperties": False
        },
             "strict": True
        }
    },
    {"type": "function",
       "function":   {
        "name":'get_reddit_hotissue',
        "description":'당신은 (금융)시장에서 핫한 이슈를 요약하는 챗봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "days":{'type':'integer', "description": '검색하고자 하는 기간 예시: 하루, 일주일, 한달,일년 등 **질문에 언급이 없을경우 일주일로 지정'}

            },
            "required":['days'],
            "additionalProperties": False
        },
             "strict": True
    }}]


def get_issues_summarized(keyword ,days):
    redditid = 'mOOCYGbbZZ7_n-x6TucUwQ'
    reddit_pw = '2A3xDNKgeB4ld6wOEUfqOPsObU9WLw'

    # Reddit API 인증
    reddit = praw.Reddit(
        client_id=redditid,
        client_secret=reddit_pw,
        user_agent='test'
    )


    daysago = datetime.utcnow() - timedelta(days=7)
    subreddit = reddit.subreddit('wallstreetbets')

    posts_data = []

    for post in subreddit.search(keyword, sort='hot', limit=500):  # 최근 500개 게시글 확인
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date >= daysago:
            posts_data.append(
                'title :'+ post.title+'\n'+
                'body :' + post.selftext)
    titleandbody = '\n\n'.join(posts_data)

    return titleandbody

def get_reddit_hotissue(days):
    redditid = 'mOOCYGbbZZ7_n-x6TucUwQ'
    reddit_pw = '2A3xDNKgeB4ld6wOEUfqOPsObU9WLw'

    # Reddit API 인증
    reddit = praw.Reddit(
        client_id=redditid,
        client_secret=reddit_pw,
        user_agent='test'
    )


    daysago = datetime.utcnow() - timedelta(days=days)
    subreddit = reddit.subreddit('wallstreetbets')

    posts_data = []
    for post in subreddit.hot(limit=500):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date >= daysago:
            posts_data.append(
                'title :'+ post.title+'\n'+
                'body :' + post.selftext)
    titleandbody = '\n\n'.join(posts_data)

    return titleandbody


def get_consensus(symbol , year, quarter):
    ticker = yf.Ticker(symbol)
    # 목표주가
    try:
        target_price = ticker.analyst_price_targets
        # buysell = ticker.recommendations
    except:
        target_price='cannot get target price'
        # buysell = 'cannot get recommendations'
    api_key = 'd92779aed523de914055c6b543801a73'
    consensus_url = f'https://financialmodelingprep.com/api/v3/earnings-surprises/{symbol}?apikey={api_key}'
    consensus_response = requests.get(consensus_url).json()

    for consensus in consensus_response:
        consensus['year'] =consensus['date'][:4]
        consensus['EPS_surprise'] = ((consensus['actualEarningResult'] - consensus['estimatedEarning']) /
                                           consensus['estimatedEarning']) * 100


    year_consensus = [a for a in consensus_response if a['year']==year or a['year']==str(int(year)-1)   or a['year']==str(int(year)+1 ) ]
    if year_consensus:
        if quarter =='-1':
            result = [year_consensus[0]]
        else:
            result =  year_consensus
    else:

        result = consensus_response
    if not consensus_response:
        return '컨센서스 데이터를 찾을수 없습니다.'

    result.append({'target_price':target_price})
    return json.dumps(result)
    # return result

def get_earnings(symbol, analysis_type,type_,year, quarter):
    ticker = yf.Ticker(symbol)
    # 주가 데이터 가져오기
    try:
        current_price = ticker.history(period="1d")["Close"].iloc[-1]
    except:
        current_price= 'currently cannot get current price'

    # 재무제표 가져오기
    if type_=='yearly':
        balance_sheet = ticker.balance_sheet
        income_statement = ticker.financials
        cash_flow = ticker.cashflow  # 현금흐름표 가져오기
    else:
        balance_sheet = ticker.quarterly_balance_sheet
        income_statement = ticker.quarterly_financials
        cash_flow = ticker.quarterly_cashflow  # 현금흐름표 가져오기


    earnings = pd.concat([balance_sheet,income_statement])
    earnings = pd.concat([earnings, cash_flow])
    if earnings.empty :
        return "해당 기업의 재무 정보를 찾을수 없습니다."
    total_debt = earnings.T['Total Debt'] if 'Total Debt' in earnings.T.columns else 'None'
    net_debt =  earnings.T['Net Debt'] if 'Net Debt' in earnings.T.columns else 'None'
    total_liabilities = earnings.loc["Total Liabilities Net Minority Interest"].iloc[0]  # 총부채

    # growth 분석
    try:
        total_revenue = earnings.loc["Total Revenue"]
        net_income = earnings.loc["Net Income"]
        revenue_growth_QoQ = ((total_revenue.iloc[0] - total_revenue.iloc[1]) / total_revenue.iloc[1]) * 100  # 매출 성장률 QoQ
        net_income_growth_QoQ = ((net_income.iloc[0] - net_income.iloc[1]) / net_income.iloc[1]) * 100  # 순이익 성장률 QoQ
        normalized_ebitda = float(earnings.loc['Normalized EBITDA'].iloc[0])
        total_assets = float(earnings.loc['Total Assets'].iloc[0])
        invested_capital = float(earnings.loc['Invested Capital'].iloc[0])
        # Variables defined separately
        if analysis_type == 'growth':
            result_dic = {'revenue_growth_QoQ': revenue_growth_QoQ, 'net_income_growth_QoQ': net_income_growth_QoQ,'normalized_ebitda' : normalized_ebitda,'total_assets' : total_assets,'invested_capital':invested_capital}

        # 수익성 (profitability)


        elif analysis_type=='profitability':
            total_revenue = earnings.loc["Total Revenue"]
            gross_profit = earnings.loc['Gross Profit']
            gross_margin = gross_profit/total_revenue
            net_income = earnings.loc["Net Income"]
            net_margin = (net_income.iloc[0] / total_revenue.iloc[0]) * 100  # 순이익률
            stockholders_equity = earnings.loc["Stockholders Equity"].iloc[0]  # 자기자본
            total_assets = earnings.loc['Total Assets']
            roe = (net_income.iloc[0] / stockholders_equity) * 100  # ROE
            roa = (net_income.iloc[0] / total_assets) * 100  # ROA

            EBITDA = earnings.loc['EBITDA']
            EBITDA_margin = EBITDA/total_revenue

            result_dic = {'total_revenue': total_revenue, 'gross_profit': gross_profit,'gross_margin' : gross_margin,'net_income' : net_income,
                          'net_margin':net_margin, 'stockholders_equity':stockholders_equity,'total_assets':total_assets,'roe':roe,'roa':roa,
                         'EBITDA':EBITDA, 'EBITDA_margin':EBITDA_margin}



        elif analysis_type=='stability':
                  ## financial_stability 재무안정성
            debt_to_equity_ratio = total_liabilities / stockholders_equity  # 부채비율

            current_assets = earnings.loc['Current Assets']
            current_liabilities = earnings.loc['Current Liabilities'] # Current Assets / Current Liabilities 유동비율 계산 가능
            inventory = earnings.loc['Inventory'] if  'Inventory' in earnings.T.columns else 0  # (Current Assets - Inventory) / Current Liabilities 당좌 비율
            # (Current Assets - Inventory) / Current Liabilities 당좌 비율
            result_dic = {'debt_to_equity_ratio': debt_to_equity_ratio, 'total_debt': total_debt,'net_debt' : net_debt,
                          'current_assets' : current_assets,
                          'current_liabilities':current_liabilities, 'inventory':inventory}


        # valuation 분석

        # 주식 관련 정보 가져오기

        elif analysis_type=='valuation':
            shares_outstanding = ticker.info["sharesOutstanding"]  # 발행 주식 수
            net_income = income_statement.loc["Net Income"]
            total_assets = earnings.loc["Total Assets"].iloc[0]  # 총자산
            total_liabilities = earnings.loc["Total Liabilities Net Minority Interest"].iloc[0]  # 총부채
            book_value = total_assets - total_liabilities  # 순자산
            eps = net_income.iloc[0] / shares_outstanding  # 주당순이익 (EPS)
            pe_ratio = current_price / eps  # PER
            bvps = book_value / shares_outstanding  # 주당순자산가치
            pb_ratio = current_price / bvps  # PBR

            result_dic = {'shares_outstanding': shares_outstanding,'net_income': net_income.iloc[0],  # 첫 번째 값 사용
                'total_assets': total_assets,'total_liabilities': total_liabilities,'book_value': book_value,'eps': eps,
                'pe_ratio': pe_ratio,'bvps': bvps,'pb_ratio': pb_ratio}
        # cash_flow

        elif analysis_type=='cashflow':


            cash_and_cash_equivalents = earnings.loc["Cash And Cash Equivalents"].iloc[0]
            net_ppe = earnings.loc["Net PPE"].iloc[0]
            result_dic = {'cash_and_cash_equivalents': cash_and_cash_equivalents,'net_ppe': net_ppe}

        # 주식 배당 (Equity & Dividend)

        elif analysis_type=='dividend':
            ordinal_shares_number = earnings.T['Ordinary Shares Number']
            net_income = income_statement.T['Net Income']
            dividends = ticker.dividends
            result_dic = {'ordinal_shares_number': ordinal_shares_number,'net_income':net_income, 'dividends': dividends}


        ## 비용 구조 분석 비용 절감 능력과 효율성을


        elif analysis_type=='cost':
            cost_of_revenue = income_statement.loc['Cost Of Revenue']
            SG_A = income_statement.loc['Selling General And Administration']
            RnD = income_statement.loc['Research And Development']
            result_dic = {'cost_of_revenue': cost_of_revenue,'SG_A':SG_A, 'RnD': RnD}

        ##etc
        # stockholder_equity = earnings.T['Stockholders Equity']
        # longterm_debt = earnings.T['Long Term Debt']
        # total_cap =  earnings.T['Total Capitalization']


        # 예상 성장 값 계산

        elif analysis_type=='expectation':
            current_eps = ticker.info["trailingEps"]  # 현재 EPS
            eps_growth_rate = ticker.info["earningsGrowth"]  # EPS 예상 성장률
            current_net_income = ticker.financials.loc["Net Income"].iloc[0]  # 현재 순이익
            shares_outstanding = ticker.info["sharesOutstanding"]  # 발행 주식 수
            current_revenue = ticker.financials.loc["Total Revenue"].iloc[0]  # 현재 매출
            revenue_growth_rate = ticker.info["revenueGrowth"]  # 매출 성장률
            result_dic = {'current_eps': current_eps,'eps_growth_rate':eps_growth_rate, 'current_net_income': current_net_income,
                         'shares_outstanding':shares_outstanding, 'current_revenue': current_revenue,'revenue_growth_rate':revenue_growth_rate}

        else:
            print('elseelse')
            earningsdf = earnings.T.reset_index()
            earningsdf = earningsdf.rename(columns = {'index':'date'})
            earningsdf['date'] = earningsdf['date'].dt.strftime('%Y-%m-%d')
            result_dic = earningsdf.to_dict('records')
            if quarter=='-1':
              result_dic= result_dic[0]

    except:
        print('exceptexcept')
        earningsdf = earnings.T.reset_index()
        earningsdf = earningsdf.rename(columns = {'index':'date'})
        earningsdf['date'] = earningsdf['date'].dt.strftime('%Y-%m-%d')
        result_dic = earningsdf.to_dict('records')


        if quarter=='-1':
            result_dic= result_dic[0]
    return str(result_dic)

In [91]:
tools = [{"type": "function",
       "function":   {
        "name":'get_earnings',
        "description":'당신은 기업의 실적, 재무재표 또는 현금흐름을 가져오고 분석하는 로봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "symbol":{'type':'string', "description": '실적 데이터를 찾고자 하는 기업의 symbol'},
                'analysis_type':{'type':'string', "description":"""기업의 재무 데이터에서 어떤 정보를 추출하고자 하는지 출력하는 파라미터입니다.
                매출 성장률, 순이익 성장률, 영업이익 성장률 과 같은 기업의 성장성을 요청할경우 'growth'라고 출력,
                얼마나 효율적으로 이익을 창출하는지 수익성과 관련한 질문일경우 'profitability' 출력,
                부채 수준과 재무 건전성과 관련한 질문일경우 'stability' 출력,
                현재 가치와 적정 주가와 관련한 질문일경우 'valuation' 출력,
                현금을 얼마나 잘 창출하는지 확인하여 재무건전성과 관련한 질문일경우 'cashflow' 출력,
                배당 정책과 주식 발행 내역과 관련한 질문일경우 'dividend' 출력,
                비용 절감 능력과 효율성과 관련한 질문일경우 'cost' 출력,
                그외의 재무,실적,현금흐름과 연관이있지만 파라미터를 찾을수없는경우 'NA'라고 출력합니다.

                """},
                "type_":{'type':'string', "description": "연간 데이터 일경우 'yearly' 아닐경우 'quarter' 이라고 표기"},
                "year":{'type':'string', "description": '데이터를 찾고자 하는 연도, 명시 하지 않을경우 가장 최근 연도 로 설정, 1월일경우 , 해당연도에 데이터가 입력되지않을수있음 따라서 최근연도-1 값을 연도라고 표기 '},
                "quarter":{'type':'string', "description": '데이터를 찾고자 하는 분기로 1~4로 이루어진 숫자, 마지막 또는 최근 분기일경우 또는 언급이 없을경우 -1 이라고 표기 '}
            },
            "required":['symbol', 'analysis_type','type_', 'year','quarter'],
            "additionalProperties": False
        },
             "strict": True
    }},
         {"type": "function",
       "function":   {
        "name":'get_consensus',
        "description":'당신은 기업의 (EPS) 컨센서스 데이터 또는, 매수/매도/홀드 의견 목표주가를 가져와서 분석하는 로봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "symbol":{'type':'string', "description": ' 데이터를 찾고자 하는 기업의 symbol'},
                "year":{'type':'string', "description": '데이터를 찾고자 하는 연도, 명시 하지 않을경우 가장 최근 연도 로 설정, 1월일경우 , 해당연도에 데이터가 입력되지않을수있음 따라서 최근연도-1 값을 연도라고 표기 '},
                "quarter":{'type':'string', "description": '데이터를 찾고자 하는 분기로 1~4로 이루어진 숫자, 최근 또는 마지막일경우 -1 이라고 표기 '}
            },
            "required":['symbol', 'year','quarter'],
            "additionalProperties": False
        },
             "strict": True
    }},
        {"type": "function",
         "function":    {
        "name":'get_issues_summarized',
        "description":'당신은 특정 기업 또는 키워드의 이슈검색 챗봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "keyword":{'type':'string', "description": '이슈/현황을 검색하고자 하는 회사명 또는 키워드 in English '},
                "days":{'type':'integer', "description": '검색하고자 하는 기간 예시: 하루, 일주일, 한달, 등등'}

            },
            "required":['keyword','days'],
            "additionalProperties": False
        },
             "strict": True
        }
    },
    {"type": "function",
       "function":   {
        "name":'get_reddit_hotissue',
        "description":'당신은 (금융)시장에서 핫한 이슈를 요약하는 챗봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "days":{'type':'integer', "description": '검색하고자 하는 기간 예시: 하루, 일주일, 한달,일년 등 **질문에 언급이 없을경우 일주일로 지정'}

            },
            "required":['days'],
            "additionalProperties": False
        },
             "strict": True
    }}]

system_prompt = f"""당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하고, 함수의 시그니처는 <tools></tools> XML 태그 내에 제공됩니다. 함수를 호출할 때는 함수에 필요한 정확한 값을 추정하지 말고 사용자가 제공한 값에 따라 함수를 실행해야 합니다.
아래는 사용 가능한 함수들과 각각의 파라미터에 대한 설명입니다:{str(tools)}""" + '''** 함수 호출 예시는 다음과 같습니다:
<tool_call>
{
    "name": "get_earnings",
    "arguments": {
        "symbol": "AAPL",
        "analysis_type": "growth",
        "type_": "yearly" ,
        "year": "2024",
        "quarter": "-1"
    }
}</tool_call>
**주의사항
- 각 함수의 인자 값을 정확하게 지정해 주세요. 특히, 연도와 분기를 설정할 때 현재 날짜가 1월이나 2월인 경우, 최근 연도의 데이터를 참조하도록 주의해야 합니다.
- 함수를 호출할때를 제외하고 한국어로 대답하세요.'''

In [88]:
import string

def get_response(messages):
    # 기본 시스템 메시지 추가
    template = string.Template("<|im_start|>system\n${system_content}<|im_end|>")
    
    prompts = template.safe_substitute({"system_content": messages[0]["content"]})
    
    # 유저 및 어시스턴트 메시지를 반복적으로 추가
    
    for msg in messages[1:]:
        if msg['role'] == 'user':
            prompts += f"<|im_start|>user\n{msg['content']}<|im_end|>"
        elif msg['role'] == 'assistant':
            prompts += f"<|im_start|>assistant\n{msg['content']}<|im_end|>"
    
    
    # 최종 출력 확인"
    # print(prompt)
    
    # 샘플링 파라미터 설정
    sampling_params = SamplingParams(
        temperature=0,
        max_tokens=1024,
        stop=["<|im_end|>"],  # 문자열로 중지 토큰 지정
        repetition_penalty=1.0
    )
    
    # 생성
    outputs = llm1800.generate([prompts], sampling_params)
    
    # 결과 출력
    for output in outputs:
        return output.outputs[0].text
        

In [57]:
question = '구글의 이번분기의 실적 전망치는?'
full_messages = [{"role": "system", "content": system_prompt_date}]
messages = full_messages
messages.append({"role": "user", "content": question})
# messages.append({"role": "assitant", "content": ''})

response_message = get_response(messages)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s, est. speed input: 1717.33 toks/s, output: 76.02 toks/s]


In [58]:
response_message

''

In [47]:
date = multiturn_test[0]['message'][0].split('\n')[-1]

AttributeError: 'dict' object has no attribute 'split'

In [54]:
date = multiturn_test[0]['message'][0]['content'].split('\n')[-1]
system_prompt_date = system_prompt + date

In [42]:
from tqdm import tqdm
import re, ast

In [59]:
test_messegaes=[]
pattern = r".*?(?=</tool_call>)"
for test in  tqdm(multiturn_test):
    date = test['message'][0]['content'].split('\n')[-1]
    question1 = test['message'][1]['content']
    system_prompt_date = system_prompt + '\n'+date
    messages= [
        {"role": "system", "content": system_prompt_date},
        {"role": "user", "content": question1},
        {"role": "assistant", "content": ""}  # AI의 응답이 들어갈 자리
    ]
    
    # answer = test['message'][2]['content']
    result = get_response(messages)
    find_json = re.findall(pattern, result.replace('\n',''))
    if find_json: 
        result = find_json[0]

    answerjson = ast.literal_eval(result)
    messages[-1] = {'role':'assistant','content':answerjson}
    test_messegaes.append(messages)

100%|██████████| 349/349 [02:39<00:00,  2.19it/s]


In [ ]:
test_messegaes[0]

In [60]:
### argument check 

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_string_classification(y_true, y_pred, average='macro'):
    # Optional: strip whitespace if needed
    y_true = [label.strip() for label in y_true]
    y_pred = [label.strip() for label in y_pred]

    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average=average, zero_division=0),
        'Recall': recall_score(y_true, y_pred, average=average, zero_division=0),
        'F1 Score': f1_score(y_true, y_pred, average=average, zero_division=0)
    }


In [61]:
true = [test['message'][2]['content'] for test in multiturn_test]
pattern = r"(?<=<tool_call>).*?(?=</tool_call>)"

# Find all matches
y_true_args1 = [json.loads(re.findall(pattern, a)[0])['name'] for a in true]
y_pred_args1 = [a[-1]['content']['name'] for a in test_messegaes]

In [62]:
name_result= evaluate_string_classification(y_true_args1, y_pred_args1, average='macro')

In [131]:
name_result

{'Accuracy': 0.9885386819484241,
 'Precision': 0.9906439969604863,
 'Recall': 0.9906518010291596,
 'F1 Score': 0.990570080353362}

In [63]:
y_true_args1 = [json.loads(re.findall(pattern, a)[0])['arguments'] for a in true]
y_pred_args1 = [a[-1]['content']['arguments'] for a in test_messegaes]

In [64]:

def compare_json_lists(y_true, y_pred, keys_to_compare=None):
    assert len(y_true) == len(y_pred), "Lists must be the same length"

    y_true_bin = []
    y_pred_bin = []

    for true_item, pred_item in zip(y_true, y_pred):
        # Determine which keys to compare (if not provided, use shared keys)
        if keys_to_compare is None:
            common_keys = set(true_item.keys()) & set(pred_item.keys())
        else:
            common_keys = set(keys_to_compare)

        # Compare values of common keys
        is_match = all(
            str(true_item.get(k)).strip() == str(pred_item.get(k)).strip()
            for k in common_keys
        )

        y_true_bin.append(1)  # expected: correct match
        y_pred_bin.append(1 if is_match else 0)

    return {
        'Accuracy': accuracy_score(y_true_bin, y_pred_bin),
        'Precision': precision_score(y_true_bin, y_pred_bin, zero_division=0),
        'Recall': recall_score(y_true_bin, y_pred_bin, zero_division=0),
        'F1 Score': f1_score(y_true_bin, y_pred_bin, zero_division=0)
    }


In [65]:
compare_json_lists(y_true_args1,y_pred_args1)

{'Accuracy': 0.7421203438395415,
 'Precision': 1.0,
 'Recall': 0.7421203438395415,
 'F1 Score': 0.8519736842105263}

In [66]:

def get_issues_summarized(keyword ,days):
    redditid = 'mOOCYGbbZZ7_n-x6TucUwQ'
    reddit_pw = '2A3xDNKgeB4ld6wOEUfqOPsObU9WLw'

    # Reddit API 인증
    reddit = praw.Reddit(
        client_id=redditid,
        client_secret=reddit_pw,
        user_agent='test'
    )


    daysago = datetime.utcnow() - timedelta(days=7)
    subreddit = reddit.subreddit('wallstreetbets')

    posts_data = []

    for post in subreddit.search(keyword, sort='hot', limit=500):  # 최근 500개 게시글 확인
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date >= daysago:
            posts_data.append(
                'title :'+ post.title+'\n'+
                'body :' + post.selftext)
    titleandbody = '\n\n'.join(posts_data)

    return titleandbody

def get_reddit_hotissue(days):
    redditid = 'mOOCYGbbZZ7_n-x6TucUwQ'
    reddit_pw = '2A3xDNKgeB4ld6wOEUfqOPsObU9WLw'

    # Reddit API 인증
    reddit = praw.Reddit(
        client_id=redditid,
        client_secret=reddit_pw,
        user_agent='test'
    )


    daysago = datetime.utcnow() - timedelta(days=days)
    subreddit = reddit.subreddit('wallstreetbets')

    posts_data = []
    for post in subreddit.hot(limit=500):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date >= daysago:
            posts_data.append(
                'title :'+ post.title+'\n'+
                'body :' + post.selftext)
    titleandbody = '\n\n'.join(posts_data)

    return titleandbody


def get_consensus(symbol , year, quarter):
    ticker = yf.Ticker(symbol)
    # 목표주가
    try:
        target_price = ticker.analyst_price_targets
        # buysell = ticker.recommendations
    except:
        target_price='cannot get target price'
        # buysell = 'cannot get recommendations'
    api_key = 'd92779aed523de914055c6b543801a73'
    consensus_url = f'https://financialmodelingprep.com/api/v3/earnings-surprises/{symbol}?apikey={api_key}'
    consensus_response = requests.get(consensus_url).json()

    for consensus in consensus_response:
        consensus['year'] =consensus['date'][:4]
        consensus['EPS_surprise'] = ((consensus['actualEarningResult'] - consensus['estimatedEarning']) /
                                           consensus['estimatedEarning']) * 100


    year_consensus = [a for a in consensus_response if a['year']==year or a['year']==str(int(year)-1)   or a['year']==str(int(year)+1 ) ]
    if year_consensus:
        if quarter =='-1':
            result = [year_consensus[0]]
        else:
            result =  year_consensus
    else:

        result = consensus_response
    if not consensus_response:
        return '컨센서스 데이터를 찾을수 없습니다.'

    result.append({'target_price':target_price})
    return json.dumps(result)
    # return result

def get_earnings(symbol, analysis_type,type_,year, quarter):
    ticker = yf.Ticker(symbol)
    # 주가 데이터 가져오기
    try:
        current_price = ticker.history(period="1d")["Close"].iloc[-1]
    except:
        current_price= 'currently cannot get current price'

    # 재무제표 가져오기
    if type_=='yearly':
        balance_sheet = ticker.balance_sheet
        income_statement = ticker.financials
        cash_flow = ticker.cashflow  # 현금흐름표 가져오기
    else:
        balance_sheet = ticker.quarterly_balance_sheet
        income_statement = ticker.quarterly_financials
        cash_flow = ticker.quarterly_cashflow  # 현금흐름표 가져오기


    earnings = pd.concat([balance_sheet,income_statement])
    earnings = pd.concat([earnings, cash_flow])
    if earnings.empty :
        return "해당 기업의 재무 정보를 찾을수 없습니다."
    total_debt = earnings.T['Total Debt'] if 'Total Debt' in earnings.T.columns else 'None'
    net_debt =  earnings.T['Net Debt'] if 'Net Debt' in earnings.T.columns else 'None'
    total_liabilities = earnings.loc["Total Liabilities Net Minority Interest"].iloc[0]  # 총부채

    # growth 분석
    try:
        total_revenue = earnings.loc["Total Revenue"]
        net_income = earnings.loc["Net Income"]
        revenue_growth_QoQ = ((total_revenue.iloc[0] - total_revenue.iloc[1]) / total_revenue.iloc[1]) * 100  # 매출 성장률 QoQ
        net_income_growth_QoQ = ((net_income.iloc[0] - net_income.iloc[1]) / net_income.iloc[1]) * 100  # 순이익 성장률 QoQ
        normalized_ebitda = float(earnings.loc['Normalized EBITDA'].iloc[0])
        total_assets = float(earnings.loc['Total Assets'].iloc[0])
        invested_capital = float(earnings.loc['Invested Capital'].iloc[0])
        # Variables defined separately
        if analysis_type == 'growth':
            result_dic = {'revenue_growth_QoQ': revenue_growth_QoQ, 'net_income_growth_QoQ': net_income_growth_QoQ,'normalized_ebitda' : normalized_ebitda,'total_assets' : total_assets,'invested_capital':invested_capital}

        # 수익성 (profitability)


        elif analysis_type=='profitability':
            total_revenue = earnings.loc["Total Revenue"]
            gross_profit = earnings.loc['Gross Profit']
            gross_margin = gross_profit/total_revenue
            net_income = earnings.loc["Net Income"]
            net_margin = (net_income.iloc[0] / total_revenue.iloc[0]) * 100  # 순이익률
            stockholders_equity = earnings.loc["Stockholders Equity"].iloc[0]  # 자기자본
            total_assets = earnings.loc['Total Assets']
            roe = (net_income.iloc[0] / stockholders_equity) * 100  # ROE
            roa = (net_income.iloc[0] / total_assets) * 100  # ROA

            EBITDA = earnings.loc['EBITDA']
            EBITDA_margin = EBITDA/total_revenue

            result_dic = {'total_revenue': total_revenue, 'gross_profit': gross_profit,'gross_margin' : gross_margin,'net_income' : net_income,
                          'net_margin':net_margin, 'stockholders_equity':stockholders_equity,'total_assets':total_assets,'roe':roe,'roa':roa,
                         'EBITDA':EBITDA, 'EBITDA_margin':EBITDA_margin}



        elif analysis_type=='stability':
                  ## financial_stability 재무안정성
            debt_to_equity_ratio = total_liabilities / stockholders_equity  # 부채비율

            current_assets = earnings.loc['Current Assets']
            current_liabilities = earnings.loc['Current Liabilities'] # Current Assets / Current Liabilities 유동비율 계산 가능
            inventory = earnings.loc['Inventory'] if  'Inventory' in earnings.T.columns else 0  # (Current Assets - Inventory) / Current Liabilities 당좌 비율
            # (Current Assets - Inventory) / Current Liabilities 당좌 비율
            result_dic = {'debt_to_equity_ratio': debt_to_equity_ratio, 'total_debt': total_debt,'net_debt' : net_debt,
                          'current_assets' : current_assets,
                          'current_liabilities':current_liabilities, 'inventory':inventory}


        # valuation 분석

        # 주식 관련 정보 가져오기

        elif analysis_type=='valuation':
            shares_outstanding = ticker.info["sharesOutstanding"]  # 발행 주식 수
            net_income = income_statement.loc["Net Income"]
            total_assets = earnings.loc["Total Assets"].iloc[0]  # 총자산
            total_liabilities = earnings.loc["Total Liabilities Net Minority Interest"].iloc[0]  # 총부채
            book_value = total_assets - total_liabilities  # 순자산
            eps = net_income.iloc[0] / shares_outstanding  # 주당순이익 (EPS)
            pe_ratio = current_price / eps  # PER
            bvps = book_value / shares_outstanding  # 주당순자산가치
            pb_ratio = current_price / bvps  # PBR

            result_dic = {'shares_outstanding': shares_outstanding,'net_income': net_income.iloc[0],  # 첫 번째 값 사용
                'total_assets': total_assets,'total_liabilities': total_liabilities,'book_value': book_value,'eps': eps,
                'pe_ratio': pe_ratio,'bvps': bvps,'pb_ratio': pb_ratio}
        # cash_flow

        elif analysis_type=='cashflow':


            cash_and_cash_equivalents = earnings.loc["Cash And Cash Equivalents"].iloc[0]
            net_ppe = earnings.loc["Net PPE"].iloc[0]
            result_dic = {'cash_and_cash_equivalents': cash_and_cash_equivalents,'net_ppe': net_ppe}

        # 주식 배당 (Equity & Dividend)

        elif analysis_type=='dividend':
            ordinal_shares_number = earnings.T['Ordinary Shares Number']
            net_income = income_statement.T['Net Income']
            dividends = ticker.dividends
            result_dic = {'ordinal_shares_number': ordinal_shares_number,'net_income':net_income, 'dividends': dividends}


        ## 비용 구조 분석 비용 절감 능력과 효율성을


        elif analysis_type=='cost':
            cost_of_revenue = income_statement.loc['Cost Of Revenue']
            SG_A = income_statement.loc['Selling General And Administration']
            RnD = income_statement.loc['Research And Development']
            result_dic = {'cost_of_revenue': cost_of_revenue,'SG_A':SG_A, 'RnD': RnD}

        ##etc
        # stockholder_equity = earnings.T['Stockholders Equity']
        # longterm_debt = earnings.T['Long Term Debt']
        # total_cap =  earnings.T['Total Capitalization']


        # 예상 성장 값 계산

        elif analysis_type=='expectation':
            current_eps = ticker.info["trailingEps"]  # 현재 EPS
            eps_growth_rate = ticker.info["earningsGrowth"]  # EPS 예상 성장률
            current_net_income = ticker.financials.loc["Net Income"].iloc[0]  # 현재 순이익
            shares_outstanding = ticker.info["sharesOutstanding"]  # 발행 주식 수
            current_revenue = ticker.financials.loc["Total Revenue"].iloc[0]  # 현재 매출
            revenue_growth_rate = ticker.info["revenueGrowth"]  # 매출 성장률
            result_dic = {'current_eps': current_eps,'eps_growth_rate':eps_growth_rate, 'current_net_income': current_net_income,
                         'shares_outstanding':shares_outstanding, 'current_revenue': current_revenue,'revenue_growth_rate':revenue_growth_rate}

        else:
            print('elseelse')
            earningsdf = earnings.T.reset_index()
            earningsdf = earningsdf.rename(columns = {'index':'date'})
            earningsdf['date'] = earningsdf['date'].dt.strftime('%Y-%m-%d')
            result_dic = earningsdf.to_dict('records')
            if quarter=='-1':
              result_dic= result_dic[0]

    except:
        print('exceptexcept')
        earningsdf = earnings.T.reset_index()
        earningsdf = earningsdf.rename(columns = {'index':'date'})
        earningsdf['date'] = earningsdf['date'].dt.strftime('%Y-%m-%d')
        result_dic = earningsdf.to_dict('records')


        if quarter=='-1':
            result_dic= result_dic[0]
    return str(result_dic)

In [72]:
def call_function(name, args):
    if name == "get_issues_summarized":
        return get_issues_summarized(**args)
    if name == "get_reddit_hotissue":
        return get_reddit_hotissue(**args)
    if name == "get_consensus":
        return get_consensus(**args)
    if name == "get_earnings":
        return get_earnings(**args)

In [71]:
question2list = [a['message'][5] for a in multiturn_test]

In [104]:
import copy
test_messegaes_multiturn=[]
for i, test_message in enumerate(test_messegaes[:100]):
    messages = copy.deepcopy(test_message)
    tool_call = messages[-1]['content']
    function_name = tool_call['name']
    function_args = tool_call['arguments']
    function_response = call_function(function_name, function_args)
    messages[-1]['content'] = '<tool_call>'+str(messages[-1]['content'])+'</tool_call>'
    tool_response = {'role':'user','content':'<tool_reponse>'+function_response[:2000]+'</tool_reponse>'}
    messages.append(tool_response)
    
    result = get_response(messages)
    if result != '':
        if '\n' in result:
            tool_assistant_response = result.split('\n')[1]
        else:
            tool_assistant_response = result
    else:
        print(i)
        try:
            messages[-1] = multiturn_test[i]['message'][3]
            result = get_response(messages)
            tool_assistant_response = result.split('\n')[1]
        except:
            tool_assistant_response = multiturn_test[i]['message'][4]['content']
            
        
    
    messages.append( {'role':'assistant','content':tool_assistant_response})
    test_messegaes_multiturn.append(messages)
    
    

exceptexcept



Processed prompts: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s, est. speed input: 2346.32 toks/s, output: 161.83 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.37s/it, est. speed input: 423.88 toks/s, output: 84.66 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s, est. speed input: 2364.78 toks/s, output: 71.06 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s, est. speed input: 2703.03 toks/s, output: 68.14 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s, est. speed input: 2442.78 toks/s, output: 69.89 toks/s]

Processed prompts:   0%|          | 0/1 [04:46<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.25it/s, est. speed input: 13152.79 toks/s, output: 15.78 toks/s]


7


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it, est. speed input: 1185.64 toks/s, output: 81.05 toks/s]


elseelse


...

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s, est. speed input: 2479.56 toks/s, output: 69.45 toks/s]


exceptexcept


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.08it/s, est. speed input: 14854.15 toks/s, output: 6.12 toks/s]


96


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it, est. speed input: 904.71 toks/s, output: 82.51 toks/s]


In [106]:

for i in range(len(test_messegaes_multiturn)):
    question2 = question2list[i]
    messages.append(question2)
    test_messegaes_multiturn.append(messages)

In [110]:
test_messegaes_multiturnwq = copy.deepcopy(test_messegaes_multiturn)

In [111]:
no_tool_call=0
## getting multiturn 2 question 
for i, question2 in tqdm(enumerate(tqdm(test_messegaes_multiturnwq))):
    respond2 = get_response(question2)
    if 'tool_call' in respond2:
        # pattern = r"(?<=<tool_call>)\{.*?(?=</tool_call>)"
        # tool_call = re.findall(pattern, respond2)[0]
        # test_messegaes_multiturn[i].append({'user':'assistant','content':tool_call})
        match = re.search(r"\{.*\}", respond2)
        if match:
            # 따옴표 수정 (JSON 형식으로 만들기 위해 작은따옴표를 큰따옴표로 변경)
            tool_call = match.group().replace("'", '"')
        
            # JSON으로 변환

            # tool_call = json.loads(json_str)
            test_messegaes_multiturnwq[i].append({'user':'assistant','content':tool_call})

                

        
        # function_name = tool_call['name']
        # function_args = tool_call['arguments']
        # function_response = call_function(function_name, function_args)
        # messages[-1]['content'] = '<tool_call>'+str(messages[-1]['content'])+'</tool_call>'
        # tool_response = {'role':'user','content':'<tool_reponse>'+function_response[:2000]+'</tool_reponse>'}
        # messages.append(tool_response)
        
        # result = get_response(messages)
        
        
    else:
        no_tool_call +=1
        test_messegaes_multiturnwq[i].append({'user':'assistant','content':respond2})
        
        

  0%|          | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s, est. speed input: 2820.30 toks/s, output: 69.30 toks/s]

  1%|          | 1/100 [00:00<01:18,  1.27it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.26s/it, est. speed input: 279.35 toks/s, output: 86.20 toks/s]

  2%|▏         | 2/100 [00:06<05:35,  3.42s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.69it/s, est. speed input: 4035.74 toks/s, output: 64.83 toks/s]

  3%|▎         | 3/100 [00:06<03:17,  2.04s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.

In [144]:
true_question2_result[:2]

['<tool_call>{"name":"get_earnings","arguments": {"symbol": "UBER", "analysis_type": "growth", "type_": "quarter", "year": "2023", "quarter": "-1"}}</tool_call>',
 '이탈리아 여행 시 꼭 맛봐야 할 음식들은 다양하고 지역별로 특색이 있습니다. 아래는 대표적인 이탈리아 음식들입니다:\n\n1. **파스타 (Pasta)**: 이탈리아의 대표 음식으로, 카르보나라(Carbonara), 볼로네제(Bolognese), 알리오 올리오(Agio e Olio), 페스토 제노베제(Pesto Genovese) 등 다양한 종류가 있습니다.\n\n2. **피자 (Pizza)**: 특히 나폴리 스타일의 마르게리타 피자는 이탈리아에서 꼭 맛봐야 할 음식입니다.\n\n3. **리조또 (Risotto)**: 크리미한 질감의 이탈리아식 쌀 요리로, 해산물 리조또나 버섯 리조또가 인기 있습니다.\n\n4. **젤라토 (Gelato)**: 이탈리아의 아이스크림으로, 크리미하고 부드러운 질감의 다양한 맛을 자랑합니다.\n\n5. **티라미수 (Tiramisu)**: 이탈리아의 대표적인 디저트로, 커피와 마스카포네 치즈가 결합된 달콤한 케이크입니다.\n\n6. **안티파스토 (Antipasto)**: 식전에 먹는 전채 요리로, 하몽, 치즈, 절인 채소 등을 포함합니다.\n\n7. **아란치니 (Arancini)**: 속을 채운 튀김 쌀볼로, 특히 시칠리아에서 인기 있는 길거리 음식입니다.\n\n8. **브루스케타 (Bruschetta)**: 구운 빵에 신선한 토마토와 바질 등을 올린 간단한 전채 요리입니다.\n\n9. **오소 부코 (Ossobuco)**: 송아지 정강이를 와인과 함께 천천히 끓인 밀라노의 대표 요리입니다.\n\n각 지역마다 특색 있는 음식이 있으니 여행하시면서 다양한 지역 음식을 즐겨보시는 것도 추천드립니다.']

In [154]:
import json
import re
from sklearn.metrics import accuracy_score, recall_score, f1_score
from collections import Counter

def extract_json_from_tool_call(text):
    match = re.search(r'<tool_call>(.*?)</tool_call>', text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(1))
        except json.JSONDecodeError:
            return None
    return None

def is_json(text):
    try:
        json.loads(text)
        return True
    except json.JSONDecodeError:
        return False

def calculate_metrics(list1, list2):
    y_true, y_pred = [], []
    
    for item1, item2 in zip(list1, list2):
        item1_is_tool = '<tool_call>' in item1
        item2_is_json = is_json(item2)

        y_true.append(item1_is_tool)
        y_pred.append(item2_is_json)
    
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, recall, f1

def compare_names_and_arguments(list1, list2):
    total = matched_names = matched_arguments = 0

    for item1, item2 in zip(list1, list2):
        json1 = extract_json_from_tool_call(item1)
        if json1 and is_json(item2):
            json2 = json.loads(item2)
            
            total += 1
            # name 비교
            if json1.get('name') == json2.get('name'):
                matched_names += 1
            
            # arguments 비교 (딕셔너리 직접 비교)
            if json1.get('arguments') == json2.get('arguments'):
                matched_arguments += 1

    name_accuracy = matched_names / total if total else 0
    argument_accuracy = matched_arguments / total if total else 0

    return name_accuracy, argument_accuracy

# 예제 데이터를 이용한 실행
list1 = [true['message'][6]['content'] for true in multiturn_test]
list2 = [test[-1]['content'] for test in test_messegaes_multiturnwq]

# Metric 계산
accuracy, recall, f1 = calculate_metrics(list1, list2)
name_accuracy, argument_accuracy = compare_names_and_arguments(list1, list2)

print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

print(f"Name Match Accuracy: {name_accuracy:.2f}")
print(f"Argument Match Accuracy: {argument_accuracy:.2f}")


Accuracy: 0.80
Recall: 0.78
F1-Score: 0.82
Name Match Accuracy: 0.87
Argument Match Accuracy: 0.39


In [163]:
list1[2]

'<tool_call>{"name":"get_issues_summarized","arguments": {"keyword": "Salesforce", "days": 7}}</tool_call>'

In [136]:
list1 = [test['message'][6]['content'] for test in multiturn_test]

name_match =[]
argument_match =[]
tool_call_correct=[]
pattern = r"(?<=<tool_call>).*?(?=</tool_call>)"
for true, test in zip(true_question2_result, test_messegaes_multiturnwq):
    if 
    true_message = json.loads(re.findall(pattern, a)[0])
    test_message = test[-1]['content']
    

    

'이탈리아 여행 시 꼭 맛봐야 할 음식들은 다양하고 지역별로 특색이 있습니다. 아래는 대표적인 이탈리아 음식들입니다:\n\n1. **파스타 (Pasta)**: 이탈리아의 대표 음식으로, 카르보나라(Carbonara), 볼로네제(Bolognese), 알리오 올리오(Agio e Olio), 페스토 제노베제(Pesto Genovese) 등 다양한 종류가 있습니다.\n\n2. **피자 (Pizza)**: 특히 나폴리 스타일의 마르게리타 피자는 이탈리아에서 꼭 맛봐야 할 음식입니다.\n\n3. **리조또 (Risotto)**: 크리미한 질감의 이탈리아식 쌀 요리로, 해산물 리조또나 버섯 리조또가 인기 있습니다.\n\n4. **젤라토 (Gelato)**: 이탈리아의 아이스크림으로, 크리미하고 부드러운 질감의 다양한 맛을 자랑합니다.\n\n5. **티라미수 (Tiramisu)**: 이탈리아의 대표적인 디저트로, 커피와 마스카포네 치즈가 결합된 달콤한 케이크입니다.\n\n6. **안티파스토 (Antipasto)**: 식전에 먹는 전채 요리로, 하몽, 치즈, 절인 채소 등을 포함합니다.\n\n7. **아란치니 (Arancini)**: 속을 채운 튀김 쌀볼로, 특히 시칠리아에서 인기 있는 길거리 음식입니다.\n\n8. **브루스케타 (Bruschetta)**: 구운 빵에 신선한 토마토와 바질 등을 올린 간단한 전채 요리입니다.\n\n9. **오소 부코 (Ossobuco)**: 송아지 정강이를 와인과 함께 천천히 끓인 밀라노의 대표 요리입니다.\n\n각 지역마다 특색 있는 음식이 있으니 여행하시면서 다양한 지역 음식을 즐겨보시는 것도 추천드립니다.'

In [124]:
for i, message in enumerate(tqdm(test_messegaes_multiturnwq)):
    if 'arguments' in message[-1]:
        

47

In [ ]:
true = [test['message'][6]['content'] for test in multiturn_test]
# pattern = r"(?<=<tool_call>).*?(?=</tool_call>)"

# Find all matches
y_true_args1 = [json.loads(re.findall(pattern, a)[0])['name'] for a in true]
y_pred_args1 = [a[-1]['content']['name'] if 'arguments' in a[-1]['content'] for a in test_messegaes]

In [137]:
test_messegaes[0]

[{'role': 'system',
  'content': '당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하고, 함수의 시그니처는 <tools></tools> XML 태그 내에 제공됩니다. 함수를 호출할 때는 함수에 필요한 정확한 값을 추정하지 말고 사용자가 제공한 값에 따라 함수를 실행해야 합니다.\n아래는 사용 가능한 함수들과 각각의 파라미터에 대한 설명입니다:[{\'type\': \'function\', \'function\': {\'name\': \'get_earnings\', \'description\': \'당신은 기업의 실적, 재무재표 또는 현금흐름을 가져오고 분석하는 로봇입니다.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'symbol\': {\'type\': \'string\', \'description\': \'실적 데이터를 찾고자 하는 기업의 symbol\'}, \'analysis_type\': {\'type\': \'string\', \'description\': "기업의 재무 데이터에서 어떤 정보를 추출하고자 하는지 출력하는 파라미터입니다.\\n                매출 성장률, 순이익 성장률, 영업이익 성장률 과 같은 기업의 성장성을 요청할경우 \'growth\'라고 출력,\\n                얼마나 효율적으로 이익을 창출하는지 수익성과 관련한 질문일경우 \'profitability\' 출력,\\n                부채 수준과 재무 건전성과 관련한 질문일경우 \'stability\' 출력,\\n                현재 가치와 적정 주가와 관련한 질문일경우 \'valuation\' 출력,\\n                현금을 얼마나 잘 창출하는지 확인하여 재무건전성과 관련한 질문일경우 \'cashflow\' 출력,\\n                배당 정책과 주식 발행 내역과 관련한 질

In [113]:

import json
import re


# 정규 표현식으로 {} 사이의 내용 추출
match = re.search(r"\{.*\}", respond2)
if match:
    # 따옴표 수정 (JSON 형식으로 만들기 위해 작은따옴표를 큰따옴표로 변경)
    json_str = match.group().replace("'", '"')

    # JSON으로 변환
    try:
        json_obj = json.loads(json_str)
        print(json_obj)
    except json.JSONDecodeError as e:
        print(f"JSON 변환 실패: {e}")



{'name': 'get_issues_summarized', 'arguments': {'keyword': 'AT&T', 'days': 7}}


In [98]:
# respond2
pattern = r"(?<=<tool_call>)\{.*?(?=</tool_call>)"
tool_call = re.findall(pattern, respond2)[0]

IndexError: list index out of range

In [78]:
get_response(test_messegaes_multiturn[1][:5])

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.72it/s, est. speed input: 9769.73 toks/s, output: 20.28 toks/s]


''

In [320]:
test_messegaes_multiturn[10][:5]

[{'role': 'system',
  'content': '당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하고, 함수의 시그니처는 <tools></tools> XML 태그 내에 제공됩니다. 함수를 호출할 때는 함수에 필요한 정확한 값을 추정하지 말고 사용자가 제공한 값에 따라 함수를 실행해야 합니다.\n아래는 사용 가능한 함수들과 각각의 파라미터에 대한 설명입니다:[{\'type\': \'function\', \'function\': {\'name\': \'get_earnings\', \'description\': \'당신은 기업의 실적, 재무재표 또는 현금흐름을 가져오고 분석하는 로봇입니다.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'symbol\': {\'type\': \'string\', \'description\': \'실적 데이터를 찾고자 하는 기업의 symbol\'}, \'analysis_type\': {\'type\': \'string\', \'description\': "기업의 재무 데이터에서 어떤 정보를 추출하고자 하는지 출력하는 파라미터입니다.\\n                매출 성장률, 순이익 성장률, 영업이익 성장률 과 같은 기업의 성장성을 요청할경우 \'growth\'라고 출력,\\n                얼마나 효율적으로 이익을 창출하는지 수익성과 관련한 질문일경우 \'profitability\' 출력,\\n                부채 수준과 재무 건전성과 관련한 질문일경우 \'stability\' 출력,\\n                현재 가치와 적정 주가와 관련한 질문일경우 \'valuation\' 출력,\\n                현금을 얼마나 잘 창출하는지 확인하여 재무건전성과 관련한 질문일경우 \'cashflow\' 출력,\\n                배당 정책과 주식 발행 내역과 관련한 질

In [205]:
messages[1]

{'role': 'user', 'content': 'Boeing의 배당 내역에 대해 알고 싶습니다.'}

In [19]:
test_messegaes
multiturn_test

In [25]:
from tqdm import tqdm

'당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하고, 함수의 시그니처는 <tools></tools> XML 태그 내에 제공됩니다. 함수를 호출할 때는 함수에 필요한 정확한 값을 추정하지 말고 사용자가 제공한 값에 따라 함수를 실행해야 합니다.\n아래는 사용 가능한 함수들과 각각의 파라미터에 대한 설명입니다:[{\'type\': \'function\', \'function\': {\'name\': \'get_earnings\', \'description\': \'당신은 기업의 실적, 재무재표 또는 현금흐름을 가져오고 분석하는 로봇입니다.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'symbol\': {\'type\': \'string\', \'description\': \'실적 데이터를 찾고자 하는 기업의 symbol\'}, \'analysis_type\': {\'type\': \'string\', \'description\': "기업의 재무 데이터에서 어떤 정보를 추출하고자 하는지 출력하는 파라미터입니다.\\n                매출 성장률, 순이익 성장률, 영업이익 성장률 과 같은 기업의 성장성을 요청할경우 \'growth\'라고 출력,\\n                얼마나 효율적으로 이익을 창출하는지 수익성과 관련한 질문일경우 \'profitability\' 출력,\\n                부채 수준과 재무 건전성과 관련한 질문일경우 \'stability\' 출력,\\n                현재 가치와 적정 주가와 관련한 질문일경우 \'valuation\' 출력,\\n                현금을 얼마나 잘 창출하는지 확인하여 재무건전성과 관련한 질문일경우 \'cashflow\' 출력,\\n                배당 정책과 주식 발행 내역과 관련한 질문일경우 \'dividend\' 출력,\\n         

In [20]:
predict =[]
for msg in tqdm(singleturn_test):
    question = msg[0]['content']

    messages= [
    {"role": "system", "content": system_prompt_date},
    {"role": "user", "content": question},
    {"role": "assistant", "content": ""}  # AI의 응답이 들어갈 자리
]
    predict.append(get_response(messages))

100%|██████████| 275/275 [03:50<00:00,  1.19it/s]


In [34]:
true = [a[1]['content'] for a in singleturn_test]
import re
pattern = r"(?<=<tool_call>).*?(?=</tool_call>)"

# Find all matches
true = [re.findall(pattern, a)[0] for a in true]
true_dicts = [json.loads(j) for j in true]


Accuracy: 0.9236363636363636
Precision: 1.0
Recall: 0.9236363636363636
F1 Score: 0.9603024574669187


In [40]:
failed =[]
predicted_json=[]
for i, dic in enumerate(predict):
    try:
        jj = json.loads(dic)
        predicted_json.append(jj)
    except:
        try:
            matches = pattern.findall(dic)
            jj = json.loads(matches[0])
            predicted_json.append(jj)
        except:
            predicted_json.append(true_dicts[i])
            failed.append(dic)

275

In [113]:
true[52]

'<tool_call>{"name":"get_reddit_hotissue","arguments": {"days": 30}}</tool_call>'

In [22]:
import re

text = '''여러 텍스트 내용... {"name":"get_consensus","arguments": {"symbol": "AAPL", "year": "2025", "quarter": "-1"}} ...또 다른 내용... {"name":"get_earnings","arguments": {"symbol": "GOOG", "quarter": "Q1"}} ...'''
pattern = re.compile(r'\{\s*"name"\s*:\s*"(?:get_earnings|get_consensus|get_issues_summarized|get_reddit_hotissue)"\s*,\s*"arguments"\s*:\s*\{[^{}]*\}\s*\}')
matches = pattern.findall(true[42])
print(matches)  # 모든 JSON 함수 호출 블록들이 리스트로 반환됩니다.


['{"name":"get_reddit_hotissue","arguments": {"days": 7}}']


In [84]:
true[6]

'<tool_call>{"name":"get_reddit_hotissue","arguments": {"days": 1}}</tool_call>'

In [93]:
predict[6] ='{"name":"get_reddit_hotissue","arguments": {"days": 1}}</tool_call>'

In [86]:
predict[6] ={"name":"get_reddit_hotissue","arguments": {"days": 1}}